<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Oslo_Norway_meteostat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
pip install meteostat

In [41]:
from meteostat import Point, Hourly
from datetime import datetime
import pandas as pd

# --- CONFIGURATION ---
# Oslo Coordinates (Central)
LAT = 59.9139
LON = 10.7522
START = datetime(2024, 1, 1)
END = datetime(2025, 1, 1)

print("Fetching data from Meteostat (Ground Stations) for Oslo...")

# 1. Define Location
location = Point(LAT, LON)

# 2. Fetch Hourly Data
# Meteostat automatically finds the nearest station (likely Oslo Blindern or Gardermoen)
data = Hourly(location, START, END)
data = data.fetch()

if not data.empty:
    # 3. Clean and Format
    df = data.reset_index()

    # Rename columns to be readable
    # temp = Temperature (C)
    # wspd = Wind Speed (km/h)
    # rad  = Solar Radiation (W/m2)
    rename_map = {
        'time': 'Time',
        'temp': 'Temperature_C',
        'wspd': 'Wind_Speed_kmh',
        'rad': 'GHI_Wm2'
    }
    df = df.rename(columns=rename_map)

    # Convert Wind Speed from km/h to m/s
    if 'Wind_Speed_kmh' in df.columns:
        df['Wind_Speed_ms'] = df['Wind_Speed_kmh'] / 3.6

    # Format Dates
    df['Date'] = df['Time'].dt.strftime('%Y-%m-%d')
    df['Hour'] = df['Time'].dt.strftime('%H:%M')

    # Select Columns
    cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms']

    # Check if Radiation exists
    if 'GHI_Wm2' in df.columns and not df['GHI_Wm2'].isna().all():
        cols.append('GHI_Wm2')
        print(" > Success: Solar Radiation data found!")
    else:
        print(" > Warning: Solar Radiation ('rad') is missing or empty.")
        print("   (Use the NASA POWER script if you strictly need Solar Data).")

    # Reorder
    df_final = df[[c for c in cols if c in df.columns]]

    print("-" * 30)
    print(df_final.head())
    print("-" * 30)

    filename = "Oslo_Meteostat_2024.csv"
    df_final.to_csv(filename, index=False)
    print(f"Success! Saved to {filename}")

else:
    print("No data found for this location/time.")

Fetching data from Meteostat (Ground Stations) for Oslo...
 > Warning: Solar Radiation ('rad') is missing or empty.
   (Use the NASA POWER script if you strictly need Solar Data).
------------------------------
         Date   Hour  Temperature_C  Wind_Speed_ms
0  2024-01-01  00:00           -3.9            7.0
1  2024-01-01  01:00           -3.9            7.5
2  2024-01-01  02:00           -4.0       6.694444
3  2024-01-01  03:00           -4.4            8.0
4  2024-01-01  04:00           -4.5       9.111111
------------------------------
Success! Saved to Oslo_Meteostat_2024.csv
